In [1]:
import numpy as np
import pandas as pd

Generate Age, Gender, Race for all agents in an NTA

In [5]:
def compute_stats(nta_df, attr, estimation_vars=['E', 'M', 'C', 'P', 'Z'], filter=True):
    if filter:
        filter_vars = ['E', 'P']
        estimation_vars = filter_vars
    stats = [nta_df['{}{}'.format(attr, var)].values for var in estimation_vars]
    
    return stats

def merge_nta_stats(nta_df, attr_vals):
    '''estimate and percentage'''
    args = [compute_stats(nta_df, attr, filter=True) for attr in attr_vals]
    ret_e, ret_p = 0, 0
    for val in args:
        e, p = val
        ret_e += e
        ret_p += p
    
    return [ret_e, ret_p]

def get_cumulative_stats(nta_df, attr_vals):
    total_estimate, total_percentage = merge_nta_stats(nta_df, attr_vals)
    return total_estimate, total_percentage

def get_gender_split(total_estimate, total_percentage, male_ratio):
    total_estimate = total_estimate.item()

    male_estimate = int(male_ratio*total_estimate)
    female_estimate = total_estimate - male_estimate

    male_percentage = male_ratio*total_percentage.item()
    female_percentage = total_percentage - male_percentage

    return_dict = {'estimate': {'male': male_estimate, 'female': female_estimate}, 'percentage': {'male': male_percentage, 'female': female_percentage}}

    return return_dict

    # return {'male': [male_estimate, male_percentage], 'female': [female_estimate, female_percentage]}

In [6]:
def get_nta_race_legacy(df, nta_id, race_mapping):
    nta_df = df[df['GeoID'] == nta_id]

    nta_race = {}
    for key in race_mapping:
        nta_race[key] = {'percentage': 0.0, 'estimate': 0.0}

        for attr_value in race_mapping[key]:
            estimate, percentage = compute_stats(nta_df, attr_value, filter=True)

            nta_race[key]['percentage'] +=  max(0, percentage[0])
            nta_race[key]['estimate'] +=  max(0, estimate[0])

    return nta_race

In [8]:
def get_nta_race(df, nta_id, race_mapping):
    nta_df = df[df['GeoID'] == nta_id]

    nta_race = {}
    for key in race_mapping:
        nta_race[key] = {}

        estimate, percentage = merge_nta_stats(nta_df, race_mapping[key])

        nta_race[key]['estimate'] = max(0, estimate[0])
        nta_race[key]['percentage'] = max(0, percentage[0])

    return nta_race

In [9]:
def get_nta_age_gender(df, nta_id, age_mapping, male_ratio=0.508):
    '''estimate, percentage'''
    nta_df = df[df['GeoID'] == nta_id]

    nta_age_gender = {}
    for key in age_mapping:
        attr_vals = age_mapping[key]

        total_estimate, total_percentage = merge_nta_stats(nta_df, attr_vals)
        if key == 'U19':
            male_ratio = compute_stats(nta_df, 'PopU18M')[-1] / 100.0 # percentage of male < 19
        if key == '65A':
            male_ratio = compute_stats(nta_df, 'Pop65plM')[-1] / 100.0 # percentage of male > 65
        
        age_gender_stats = get_gender_split(total_estimate, total_percentage, male_ratio=male_ratio)

        nta_age_gender[key] = age_gender_stats

    return nta_age_gender

In [10]:
nta_demographics_file = '../nta/demo_2021acs5yr_nta.xlsx'

df = pd.read_excel(nta_demographics_file)
NTA_ID = 'BK0101'

In [11]:
race_mapping = {'hispanic': ['Hsp1'], 'white': ['WtNH'], 'black': ['AsnNH'], 'black': ['BlNH'], 
                'native': ['NHPINH', 'AIANNH'], 'other': ['OthNH', 'Rc2plNH'], 'asian': ['AsnNH']}

# race data
nta_race = get_nta_race(df, NTA_ID, race_mapping)

In [12]:
age_mapping = {'U19': ['PopU5', 'Pop5t9', 'Pop10t14', 'Pop15t19'], '20t29': ['Pop20t24', 'Pop25t29'],  
                '30t39': ['Pop30t34', 'Pop35t39'], '40t49': ['Pop40t44', 'Pop45t49'], 
                '50t64': ['Pop50t54', 'Pop55t59', 'Pop60t64'], '65A': ['Pop65t69', 'Pop70t74', 'Pop75t79','Pop80t84', 'Pop85pl']}

# age data
nta_age_gender = get_nta_age_gender(df, NTA_ID, age_mapping)

In [13]:
p = 0
for key in nta_race:
    p += nta_race[key]['estimate']
print(p)

37518


In [14]:
val = 0
for key in nta_age_gender:
    val = val + nta_age_gender[key]['male'][0] + nta_age_gender[key]['female'][0]

print(val)

37518
